IMPORTAÇÃO DE MODULOS

In [59]:
from config_path import *
import pandas as pd
import numpy as np
import glob
import re
import os

E - EXTRAÇÃO

In [66]:
divergente = r'c:\WS_OLIVEIRA\BASE_DADOS\BASE_1733\DIVERGENCIA'
tempo = r'c:\WS_OLIVEIRA\BASE_DADOS\BASE_1733\TEMPO'
acumulado = r'c:\WS_OLIVEIRA\BASE_DADOS\ACUMULADOS\consolidado_inv.xlsx'

df_acum = pd.read_excel(acumulado)
dir_div = glob.glob(os.path.join(divergente, "*.xls*"))
QTDE_DIV = len(dir_div)
dir_temp = glob.glob(os.path.join(tempo, "*.xls*"))
QTDE_TEMP = len(dir_div)


In [69]:
def loop_df(dir, val):

    list_temporaria = []
    if val == 0:
        for files in dir:
            nome_arquivo = os.path.basename(files)      
            delimitador_1 = os.path.splitext(nome_arquivo)[0]
            
            if nome_arquivo not in df_acum['NAME_FILES']:
                continue
            df_temporariro = pd.read_excel(files)
            delimitador_11 = delimitador_1.split("_")[0]
            delimitador_12 = delimitador_1.split("_")[1]

            df_temporariro['INV'] = int(delimitador_11)
            df_temporariro['CONTAGEM'] = int(delimitador_12)
            df_temporariro['NAME_FILES'] = nome_arquivo
            
            list_temporaria.append(df_temporariro)
        if not list_temporaria:
            return None    
        df_final = pd.concat(list_temporaria, ignore_index= False)
        return df_final

T - TRATAMENTO

In [70]:
# df_div = loop_df(dir_div, 1)

df_temp = loop_df(dir_temp, 0)
df_temp['tempo_gasto'] = df_temp['Fim cont.'] - df_temp['Inicio Cont.']

novas_colunas = df_temp['Contador'].str.split(' - ', expand= True)
novas_colunas.columns =['COD_FUNC', 'FUNC']

df_temp['COD_FUNC'] = novas_colunas['COD_FUNC']
df_temp['FUNC'] = novas_colunas['FUNC']
df_temp['DATA'] = pd.to_datetime(df_temp['Inicio Cont.'], format= '%d-%m-%Y').dt.normalize()

TypeError: 'NoneType' object is not subscriptable

CONSULTAS

In [63]:
caso = df_temp.copy()
caso_1 = caso.groupby(['DATA']).agg(
    end_contado = ('End. contados', 'sum'),
    tempo = ('tempo_gasto', 'sum')
).reset_index().sort_values(by='DATA', ascending= True)

caso_2 = caso.groupby(['DATA', 'CONTAGEM']).agg(
    end_contado = ('End. contados', 'sum'),
    tempo = ('tempo_gasto', 'sum')
).reset_index().sort_values(by='DATA', ascending= True)

display(df_temp.dtypes)

display(caso_2.loc[caso_2['DATA'] == '2025-11-07'])

# df_fim = df_temp[['DATA','COD_FUNC','FUNC', 'NAME_FILES', 'CONTAGEM','INV', 'Inicio Cont.', 'Fim cont.','End. contados']]
# df_fim.to_excel(acumulado, index= False, sheet_name= "tempo_gasto")

OS                          int64
Contador                   object
End. OS                     int64
End. contados               int64
End. digitados              int64
Inicio Cont.       datetime64[ns]
Fim cont.          datetime64[ns]
Inicio dig.        datetime64[ns]
Fim dig.           datetime64[ns]
INV                        object
CONTAGEM                   object
NAME_FILES                 object
tempo_gasto       timedelta64[ns]
COD_FUNC                   object
FUNC                       object
DATA               datetime64[ns]
dtype: object

,DATA,CONTAGEM,end_contado,tempo
6,2025-11-07,2,24,0 days 00:27:26
7,2025-11-07,3,15,0 days 00:21:29
